In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('../data/input/train.csv', parse_dates=['fecha_dato'])
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 47 columns):
fecha_dato               datetime64[ns]
ncodpers                 int64
ind_empleado             int64
pais_residencia          int64
sexo                     int64
age                      int64
fecha_alta               int64
ind_nuevo                int64
antiguedad               int64
indrel                   int64
ult_fec_cli_1t           int64
indrel_1mes              int64
tiprel_1mes              int64
indresi                  int64
indext                   int64
conyuemp                 int64
canal_entrada            int64
indfall                  int64
tipodom                  int64
cod_prov                 int64
ind_actividad_cliente    int64
renta                    float64
segmento                 int64
ind_ahor_fin_ult1        int64
ind_aval_fin_ult1        int64
ind_cco_fin_ult1         int64
ind_cder_fin_ult1        int64
ind_cno_fin_ult1         int64
ind_ct

In [3]:
test = pd.read_csv('../data/input/test.csv', parse_dates=['fecha_dato'])
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929615 entries, 0 to 929614
Data columns (total 23 columns):
fecha_dato               929615 non-null datetime64[ns]
ncodpers                 929615 non-null int64
ind_empleado             929615 non-null int64
pais_residencia          929615 non-null int64
sexo                     929615 non-null int64
age                      929615 non-null int64
fecha_alta               929615 non-null int64
ind_nuevo                929615 non-null int64
antiguedad               929615 non-null int64
indrel                   929615 non-null int64
ult_fec_cli_1t           929615 non-null int64
indrel_1mes              929615 non-null int64
tiprel_1mes              929615 non-null int64
indresi                  929615 non-null int64
indext                   929615 non-null int64
conyuemp                 929615 non-null int64
canal_entrada            929615 non-null int64
indfall                  929615 non-null int64
tipodom                  929615 no

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split


In [5]:
import datetime
thre_date = datetime.datetime(2016, 3, 28)

In [26]:
target_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1',
               'ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
               'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1',
               'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
               'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1',
               'ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
               'ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1',
               'ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1'] 

In [7]:
target_hist = (train[train['fecha_dato'] < thre_date]
               .groupby('ncodpers')[target_cols].sum()).reset_index().astype(int)

In [8]:
train_new = train[train['fecha_dato'] >= thre_date]

In [9]:
train_compressed = train_new.merge(target_hist, how='left', on='ncodpers', suffixes=['', '_hist'])
train_compressed = train_compressed.fillna(0)

In [10]:
y = train_compressed[target_cols].values
X = train_compressed.drop(['fecha_dato', 'ncodpers'], axis=1).drop(target_cols, axis=1).values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, verbose=1)
rf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  6.2min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=1, warm_start=False)

In [13]:
# multi_clf = OneVsRestClassifier(estimator=rf, n_jobs=1)
# multi_clf.fit(X_train, y_train)

In [14]:
y_pred = rf.predict(X_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   18.4s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:  1.1min finished


In [15]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
print 'accuracy: ', accuracy_score(y_test, y_pred)
print 'f1:', f1_score(y_test, y_pred, average='macro')
print 'precision:', precision_score(y_test, y_pred, average='macro')
print 'recall:', recall_score(y_test, y_pred, average='macro')

accuracy:  0.939082792429
f1: 0.945955222349
precision: 0.974179258663
recall: 0.925284920271


In [16]:
print 'accuracy: ', accuracy_score(y_test, y_pred)
print 'f1:', f1_score(y_test, y_pred, average='micro')
print 'precision:', precision_score(y_test, y_pred, average='micro')
print 'recall:', recall_score(y_test, y_pred, average='micro')

accuracy:  0.939082792429
f1: 0.96794208639
precision: 0.97056174211
recall: 0.965336534095


In [17]:
test_new = test.merge(target_hist, how='left', on='ncodpers', suffixes=['', '_hist'])
test_new = test_new.fillna(0)

In [18]:
X_sub = test_new.drop(['fecha_dato', 'ncodpers'], axis=1).values

In [54]:
y_sub = np.array(rf.predict_proba(X_sub))

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   17.7s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:   45.6s finished


In [29]:
last_month = train[train['fecha_dato'] == datetime.datetime(2015, 5, 28)][target_cols + ['ncodpers']]

In [31]:
last_month = test[['ncodpers', 'fecha_dato']].merge(last_month, how='left', on='ncodpers')
last_month = last_month.fillna(0).astype(int).drop(['fecha_dato'], axis=1)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [32]:
target_last_month = last_month.drop('ncodpers', axis=1).values

In [61]:
y_sub = np.array(y_sub)[:,:,1].T

In [63]:
y_sub[0].shape

(24,)

In [65]:
import numpy as np
filename = '../data/output/sub_rf_{}.csv'.format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M"))
with open(filename, 'w+') as f:
    f.write('ncodpers,added_products\n')
    for ids, res, tar in zip(test['ncodpers'], y_sub, target_last_month):
        index = np.argsort(res)[::-1][:7]
        tar_idx = np.arange(1, 25)[tar.astype(bool)]
        line = ''
        for idx in index:
            if idx not in tar_idx:
                line += ' ' + str(target_cols[idx])
#             line = ' '.join(target_cols[result.astype(bool)])
        f.write(str(ids)+','+line + '\n')

In [66]:
features = train_compressed.drop(['fecha_dato', 'ncodpers'], axis=1).drop(target_cols, axis=1).columns

In [68]:
index = np.argsort(rf.feature_importances_)[::-1]

In [69]:
print features[index]

Index([u'ind_cco_fin_ult1_hist', u'ind_ctop_fin_ult1_hist',
       u'ind_recibo_ult1_hist', u'ind_nom_pens_ult1_hist',
       u'ind_cno_fin_ult1_hist', u'ind_ecue_fin_ult1_hist',
       u'ind_nomina_ult1_hist', u'ind_reca_fin_ult1_hist', u'fecha_alta',
       u'ind_ctpp_fin_ult1_hist', u'ind_tjcr_fin_ult1_hist', u'antiguedad',
       u'ind_dela_fin_ult1_hist', u'age', u'ind_actividad_cliente',
       u'ind_valo_fin_ult1_hist', u'renta', u'ind_fond_fin_ult1_hist',
       u'cod_prov', u'canal_entrada', u'segmento', u'tiprel_1mes',
       u'ind_ctma_fin_ult1_hist', u'ind_plan_fin_ult1_hist',
       u'ind_hip_fin_ult1_hist', u'ind_ctju_fin_ult1_hist', u'sexo',
       u'ind_viv_fin_ult1_hist', u'ind_pres_fin_ult1_hist', u'ind_nuevo',
       u'indext', u'ind_deme_fin_ult1_hist', u'ind_deco_fin_ult1_hist',
       u'ult_fec_cli_1t', u'indrel', u'ind_cder_fin_ult1_hist',
       u'pais_residencia', u'indrel_1mes', u'indfall', u'ind_empleado',
       u'ind_ahor_fin_ult1_hist', u'indresi', u'conyu